In [1]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [2]:
secrets_resolver = SecretsResolverDemo()

In [3]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [4]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Receiving the OOB message

In [5]:
#oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
print(oob_url)

http://127.0.0.1:8000?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiODdlZWRiMDItN2ExMi00Mzk5LWEzODgtNjAxNjNjYmQ4MTdlIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNjeHQzdGZkUHB5NEN2c0pZSFk5WmVZaGdFRzkxVGZCMW15SlAzQmQ3TThKWS5WejZNa2tVM1hNREZHa0RMSEJ6NlZkM2Vmakp3RURzQ1NHeWNjckJxTUJEQzNuR0JvLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVnaXN0ZXIiLCJnb2FsIjoiUmVnaXN0ZXJpbm1lZGlhdG9yIiwiYWNjZXB0IjpbImRpZGNvbW0vdjIiLCJkaWRjb21tL2FpcDI7ZW52PXJmYzU4NyJdfX0


In [6]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '87eedb02-7a12-4399-a388-60163cbd817e', 'from': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'register', 'goal': 'Registerinmediator', 'accept': ['didcomm/v2', 'didcomm/aip2;env=rfc587']}}


After Bob checks the invitation, he is able to prepare a response back. Since Bob has not established a connection with Alice before, he needs to create a DID peer to be used in all communications with Alice. This DID Peer will be dedicated to Alice and only Alice. If Bob needs to comunicate with someone else, he should create a new DID Peer.

In [7]:
bob_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/bob")
print("Bob's DID:", bob_did)

Bob's DID: did:peer:2.Ez6LSfuoo14TaecmowydevTpGfrkdvjBEjC1VroNREMFPta8x.Vz6MkkSiAwiux1vmSR739zY7hTJ9RDJ9jPo7n34VTcPS49UQH.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ib2IiLCJhIjpbImRpZGNvbW0vdjIiXX0


Bob's response will depends on the `goal` code of the invitation received. Here is a response message as an example:

In [86]:
# bob_response_message = Message(
#     body = { "response_requested": True },
#     custom_headers = [{"return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/trust-ping/2.0/ping",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]]
# )
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all",
#         "created_time": int(datetime.datetime.now().timestamp())            
#                       }],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/basicmessage/2.0/message",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]],
#     body = {"content": "Who won the soccer world cup in 1986?"}
# )
# bob_response_message = Message(
#     custom_headers = [{"return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/coordinate-mediation/2.0/keylist-query",
#     body = {}
# )
# bob_response_message = Message(
#     custom_headers = [{"return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
#     body = {
#         "updates": [
#             {
#             "recipient_key": "did:key:z6MkpTHR8VNsBxYAAWHut2Geadd9jSwuBV8xRoAnwWsdvktH2",
#             "action": "remove"
#             }
#         ]
            
#     }
# )
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/discover-features/2.0/queries",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]],
#     body = {
#         "queries": [
#             { "feature-type": "protocol", "match": "https://didcomm.org/basicmessage/2.*" }
#         ]
#     }
# )
bob_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/shorten-url/1.0/request-shortened-url",
    frm = bob_did,
    to = [received_msg_decoded["from"]],
    body = {
            "url": "https://my.example.com/superlongurl",
            "requested_validity_seconds": 86400,
            "goal_code": "shorten.oobv2",
#             "short_url_slug": "algo"
    }
)
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/shorten-url/1.0/invalidate-shortened-url",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]],
#     body = {
#             "shortened_url": "http://127.0.0.1:8000/qr?_oobid=eaf4166322d04ddba2829f0807b6a7cf",
#     }
# )


In [87]:
bob_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_response_message,
    frm = bob_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Mediator

In [88]:
alice_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
print(alice_did_doc)

{'id': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'authentication': [{'id': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo', 'type': 'Ed25519VerificationKey2020', 'controller': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'publicKeyMultibase': 'z6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo'}], 'keyAgreement': [{'id': 'did:peer:2.Ez6LScxt3tfdPpy4CvsJYHY9ZeYhgEG91TfB1myJP3Bd7M8JY.Vz6MkkU3XMDFGkDLHBz6Vd3efjJwEDsCSGyccrBqMBDC3nGBo.SeyJpZCI6Im5ldy1pZCIsI

From the DID Document, Bob can understand how Alice is expecting to receive messages. In this case, he gets Alice's endpoint:

In [89]:
alice_endpoint = alice_did_doc["service"][0]["serviceEndpoint"]
print(alice_endpoint)

http://127.0.0.1:8000


Using an `https` [transport](https://identity.foundation/didcomm-messaging/spec/#transports) Bob can simply `POST` the message to the endpoint with the message in the `body` and the media type header set to `application/didcomm-encrypted+json`.

In [90]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(alice_endpoint, headers=headers, data = bob_packed_msg.packed_msg)
print(resp.json())


{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTIxV1MwdDJlbUZ0VFZwdFF6Z3pSRlJDVlVGWlpUVk9ZVEpoWTFKdmJWbHBSMUpLV0d0TFVXOTVlVXRVTGxaNk5rMXJibWN6WW10bVFrVmxObWxOVG5CWmMzRlZTM2xSVEZGWmEydHdZVUZoVGtwcVRVNTVaWE4zYlZOS2JtRXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNUMms0ZGsxVVNUTk1ha0YxVFVNMGVFOXFaM2ROUkVGcFRFTkthRWxxY0dKSmJWSndXa2RPZG1KWE1IWmtha2xwV0Znd0l6Wk1VMjFXUzB0MmVtRnRUVnB0UXpnelJGUkNWVUZaWlRWT1lUSmhZMUp2YlZscFIxSktXR3RMVVc5NWVVdFUiLCJhcHYiOiJFeDktZzRVN0xkS0tQNjVnSmdKb3JXT05Hc1BaMzNtN3FTSTlBQVQ2dkJZIiwic2tpZCI6ImRpZDpwZWVyOjIuRXo2TFNtVktLdnphbU1abUM4M0RUQlVBWWU1TmEyYWNSb21ZaUdSSlhrS1FveXlLVC5WejZNa25nM2JrZkJFZTZpTU5wWXNxVUt5UUxRWWtrcGFBYU5Kak1OeWVzd21TSm5hLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCM2TFNtVktLdnphbU1abUM4M0RUQlVBWWU1TmEyYWNSb2

In a real scenario, you will receive a success http response code in the range of 2XX such as a 202. This case will fail since it's not a real endpoint.

### Step 5: Alice responding back to Bob with a rotated DID
Alice has finally received a response back from Bob at her endpoint. The encrypted message is in the body of the POST request that can be unpacked and decrypted with the following code:

In [91]:
bob_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

Note that we also passed the resolver config as before.

Finally, Alice can see Bob's response message:

In [92]:
print(bob_unpack_msg)

UnpackResult(message=Message(id='db3f1323-f6a5-4d04-8ee9-3cc5be297ac3', type='https://didcomm.org/shorten-url/1.0/shortened-url', body={'shortened_url': 'http://127.0.0.1:8000/qr?_oobid=e7fbb7816a4b4224a365aa33b50dffd8', 'expires_time': 1662929881}, frm=None, to=None, created_time=None, expires_time=None, from_prior=None, please_ack=None, ack=None, thid='d33f4c9b-6dc1-4edc-8cde-979219673632', pthid=None, attachments=None, custom_headers=None), metadata=Metadata(encrypted=True, authenticated=True, non_repudiation=False, anonymous_sender=False, re_wrapped_in_forward=False, encrypted_from='did:peer:2.Ez6LSmVKKvzamMZmC83DTBUAYe5Na2acRomYiGRJXkKQoyyKT.Vz6Mkng3bkfBEe6iMNpYsqUKyQLQYkkpaAaNJjMNyeswmSJna.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6LSmVKKvzamMZmC83DTBUAYe5Na2acRomYiGRJXkKQoyyKT', encrypted_to=['did:peer:2.Ez6LSfuoo14TaecmowydevTpGfrkdvjBEjC1VroNREMFPta8x.Vz6MkkSiAwiux1vmSR739zY7hTJ9RDJ9jPo7n34VTcPS49UQH.SeyJpZCI6Im5ldy1pZCIsI

And also she can get Bob's DID Peer:

In [93]:
print(bob_unpack_msg.message.body)

{'shortened_url': 'http://127.0.0.1:8000/qr?_oobid=e7fbb7816a4b4224a365aa33b50dffd8', 'expires_time': 1662929881}


and from Bob's DID, she can resolve the DID Document and get Bob's service endpoint:

In [138]:
bob_did_doc = json.loads(peer_did.resolve_peer_did(bob_unpack_msg.message.frm))
bob_endpoint = bob_did_doc["service"][0]["serviceEndpoint"]
print(bob_endpoint)

MalformedPeerDIDError: Invalid peer DID provided. Does not match peer DID regexp.

Now Alice is able to respond back to Bob's. However, in order to keep DID Peers private between them, Alice must replace the DID used in the Out of Band message by a new one. That process is call [DID rotation](https://identity.foundation/didcomm-messaging/spec/#did-rotation).

A DID is rotated by sending a message to Bob including the `from_prior` header containng a JWT with the new DID in the `sub` and the prior DID in the `iss` fields.
First, she need to create the new DID Peer:

In [56]:
alice_did_new = await create_peer_did(1, 1, service_endpoint="https://www.example.com/alice")
print("Alice's NEW DID:", alice_did_new)

Alice's NEW DID: did:peer:2.Ez6LSnn1bdY7Zy5WLuXxMQWEpDb2o9L9g8fW9Z2NWdASTAAKd.Vz6MkqFwmMSecezdhHHGMQJPEpjkoSFNVBQRnQG15P1VbgJsN.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9hbGljZSIsImEiOlsiZGlkY29tbS92MiJdfQ


Then, with the help of the library she can easily creat the `from_prior` header. More details on how to make the JWT can be found in Chapter 5.3 of [DIDComm Messaging Specification](https://identity.foundation/didcomm-messaging/spec/#did-rotation)

In [57]:
from_prior = FromPrior(iss=alice_did_oob, sub=alice_did_new)

In [58]:
alice_rotate_did_message = Message(
    body = {"msg": "I'm rotating my peer DID"},
    id = "unique-id-293e9a922e",
    type = "my-protocol/1.0",
    frm = alice_did_new,
    to = [bob_unpack_msg.message.frm],
    from_prior = from_prior
)

As in previous steps, Alice will encrypt and pack the message, and POST it to Bob's endpoint:

In [59]:
alice_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_rotate_did_message,
    frm = alice_did_new,
    to = bob_unpack_msg.message.frm,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [60]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(bob_endpoint, headers=headers, data = alice_packed_msg.packed_msg)

Bob will receive the message in his endpoint, unpack and decrypt, and get the new Alice's DID. He must store Alice's new DID and use it in subsequent communications.

### Step 6: What's next --> Protocols

Now Alice and Bob have a private way to communicate. This communication channel can be used whenever they need. Normally, messages passed back and forth will follow a protocol that can be understood by both. We won't cover protocols in this tutorial. You can read more about protocols in Chapter 9 of the [DIDComm Messaging specification](https://identity.foundation/didcomm-messaging/spec/#protocols)

### Step 7: Ending a relationship

Alice and Bob can use the channel forever. They know how to pass messages and even how to rotate a DID if needed. If for any reason, Alice (or Bob) need to end the relationship, she can simply send a message rotating the DID to nothing. That is achieved by ommiting the `sub` in the `from_prior` and sending the message without a `from` attribute of the message as it's shown below:

In [61]:
from_prior_end = FromPrior(iss=alice_did_new, sub=None)

In [62]:
alice_end_message = Message(
    body = {"msg": "I'm finishing this relationship :("},
    id = "unique-id-25359a955e",
    type = "my-protocol/1.0",
    to = [bob_unpack_msg.message.frm],
    from_prior = from_prior_end
)

In [64]:
# This code fails. Will open an issue in sicpa library
alice_end_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_end_message,
    to = bob_unpack_msg.message.frm,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

DIDCommValueError: FromPrior structure is invalid: FromPrior(iss='did:peer:2.Ez6LSnn1bdY7Zy5WLuXxMQWEpDb2o9L9g8fW9Z2NWdASTAAKd.Vz6MkqFwmMSecezdhHHGMQJPEpjkoSFNVBQRnQG15P1VbgJsN.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9hbGljZSIsImEiOlsiZGlkY29tbS92MiJdfQ', sub=None, aud=None, exp=None, nbf=None, iat=None, jti=None)

In [37]:
mongo = MongoClient()

In [38]:
db = mongo.mediator
secrets = db.secrets

In [46]:
for doc in wallet.find():
    print(doc)

{'_id': 'acapy', 'mnemonic': ['east', 'gun', 'drama', 'friend', 'all', 'mechanic', 'kind', 'cattle', 'orient', 'affair', 'valley', 'old', 'hammer', 'invite', 'scrap', 'tattoo', 'monster', 'magnet', 'grab', 'cook', 'track', 'drink', 'grief', 'rose'], 'passphrase': '', 'dids': [{'alias': 'did1', 'didIdx': 0, 'uriCanonical': 'did:prism:43968aa4cabb40ccd904d2e8b56c999222ca9581d07cdc8e37298ea6085afb0a', 'uriLongForm': 'did:prism:43968aa4cabb40ccd904d2e8b56c999222ca9581d07cdc8e37298ea6085afb0a:Cj8KPRI7CgdtYXN0ZXIwEAFKLgoJc2VjcDI1NmsxEiECq06PYZiJ-1fx4pELnqj0Hsf9KSIuFmexgwfYqx-XY_Q', 'operationHash': '', 'keyPairs': [{'keyId': 'master0', 'keyTypeValue': 1, 'didIdx': 0, 'keyDerivation': 0, 'keyIdx': 0, 'privateKey': 'c6ce6cb1648c1cee9eeb5784b8c0f179a70f63942cd6670bc59c2395af4c5b2b', 'publicKey': '04ab4e8f619889fb57f1e2910b9ea8f41ec7fd29222e1667b18307d8ab1f9763f4d507258891f8a1501502cce75a1de91597ee40a570c041dee63c2f98dc136ffe', 'revoked': False}]}, {'alias': 'did2', 'didIdx': 1, 'uriCanonical': 

In [101]:

kidsDB = secrets.find({},{ id: 0, 'kid': 1})
print(kidsDB)

In [68]:
print(kidsDB)

In [74]:
kids = [k["kid"] for k in kidsDB]

In [75]:
print(kids)

['did:peer:2.Ez6LSewwgLJL7yfP4vdpefpguZ1VxcQFc55JC4fxVMJMJvXE8.Vz6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9', 'did:peer:2.Ez6LSewwgLJL7yfP4vdpefpguZ1VxcQFc55JC4fxVMJMJvXE8.Vz6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6LSewwgLJL7yfP4vdpefpguZ1VxcQFc55JC4fxVMJMJvXE8', 'did:peer:2.Ez6LSqkt71TTzTvZGbtxBRFXttz6p6QvMoVdE9ZcYEvBT59kP.Vz6MkkXgcZNWNFoj262dV92WwKx4bW2hoz26B5JwMRx6CWiM3.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MkkXgcZNWNFoj262dV92WwKx4bW2hoz26B5JwMRx6CWiM3', 'did:peer:2.Ez6LSqkt71TTzTvZGbtxBRFXttz6p6QvMoVdE9ZcYEvBT59kP.Vz6MkkXgcZNWNFoj262dV92WwKx4bW2hoz26B5JwMRx6CWiM3.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6LSqkt71TTzTv

In [202]:
jj = secrets.find({"kid": "did:peer:2.xEz6LSewwgLJL7yfP4vdpefpguZ1VxcQFc55JC4fxVMJMJvXE8.Vz6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9"})

In [201]:
try:
    print(jj[0])
except:
    print(None)

{'_id': ObjectId('62cb31efe70d6b2ebbd1f33b'), 'kid': 'did:peer:2.Ez6LSewwgLJL7yfP4vdpefpguZ1VxcQFc55JC4fxVMJMJvXE8.Vz6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9', 'type': 1, 'verification_material': {'format': 1, 'value': '{"crv":"Ed25519","x":"0EHsc_KGb6Kp8nu0rN4-wJBr1tNeWMkDPptvKWSrsHI","d":"Rzt1dmc_M2_yA6mv3Xb7u2si1o12H4nMCzO-UBhwFoo","kty":"OKP","kid":"did:peer:2.Ez6LSewwgLJL7yfP4vdpefpguZ1VxcQFc55JC4fxVMJMJvXE8.Vz6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0#6MktUD9eF8hxnJwod7jG4YouiKgUWDTMxu5KY9PiT3CLkx9"}'}}


In [193]:
print(len(list(jj)))

1


In [203]:
print(jj[0])

IndexError: no such item for Cursor instance

In [204]:
import datetime
seconds_since_epoch = datetime.datetime.now().timestamp()

In [207]:
int(datetime.datetime.now().timestamp())*1000

1657496545000

In [234]:
list(db.oobs.find())

[{'_id': ObjectId('62cb74db21ae347cd311c803'),
  'did': 'did:peer:2.Ez6LSted4CfJfxEK9u6jrs8yF5JcoKGCBy5ViwPyVoYQvLbvw.Vz6MkgFiudPe33rrExzMpp7ncJXRpMdyNP8d781wkiuMUCBS8.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0',
  'date': 1657500891000,
  'url': 'https://mediator.rootsid.cloud'}]

In [217]:
if not a:
    print("xxx")

In [232]:
db.oobs.delete_many({"url": 'https://mediator.rootsid.cloud'})

In [455]:
db.oobs.find_one()

{'_id': ObjectId('62cb74db21ae347cd311c803'),
 'did': 'did:peer:2.Ez6LSted4CfJfxEK9u6jrs8yF5JcoKGCBy5ViwPyVoYQvLbvw.Vz6MkgFiudPe33rrExzMpp7ncJXRpMdyNP8d781wkiuMUCBS8.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0',
 'date': 1657500891000,
 'url': 'https://mediator.rootsid.cloud'}

In [20]:
list(db.oobs.find({}))

[{'_id': ObjectId('62cb74db21ae347cd311c803'),
  'did': 'did:peer:2.Ez6LSted4CfJfxEK9u6jrs8yF5JcoKGCBy5ViwPyVoYQvLbvw.Vz6MkgFiudPe33rrExzMpp7ncJXRpMdyNP8d781wkiuMUCBS8.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0',
  'date': 1657500891000,
  'url': 'https://mediator.rootsid.cloud'}]

In [524]:
if db.connections.find_one({"remote_did": "asd"}):
    print('existe')
    

In [67]:
list(db.connections.find())


[{'_id': ObjectId('62cf6c28d1c49194e0decd1c'),
  'remote_did': 'did:peer:2.Ez6LStqaZmWSbAZgG1VDTMbu2PXXUCr8v5uv2Wwb1RpB9cY8S.Vz6Mkhwsd3rt8CA4ySHeFF4UBDkW38HDKxWWjEcUwKZomwgCw.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ib2IiLCJhIjpbImRpZGNvbW0vdjIiXX0',
  'local_did': 'did:peer:2.Ez6LSted4CfJfxEK9u6jrs8yF5JcoKGCBy5ViwPyVoYQvLbvw.Vz6MkgFiudPe33rrExzMpp7ncJXRpMdyNP8d781wkiuMUCBS8.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0'},
 {'_id': ObjectId('62cf6c7ad1c49194e0decd64'),
  'remote_did': 'did:peer:2.Ez6LSdAqtkD4D9xhxoQgckib5qzCKYgX3xYrPoPBdpLCJof2m.Vz6MkphgQ7pTXHDwnk87j9UzwmWD4D9v39RF8QqmdHmj1cknN.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ib2IiLCJhIjpbImRpZGNvbW0vdjIiXX0',
  'local_did': 'did:peer:2.Ez6LSted4CfJfxEK9u6jrs8yF5JcoKGCBy5ViwPyVoYQvLbvw.Vz6MkgFiudPe33rrExzMpp7ncJXRpMdyNP8d781wkiuMUCBS8.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZ

In [551]:
if not None:
    print("true")

true


In [662]:
db.connections.delete_many({})

In [28]:
int(datetime.datetime.now().timestamp()) 

1657803732

In [110]:
question = urllib.parse.quote("What Rodolfo means")
ans = requests.get("http://api.wolframalpha.com/v1/result?i="+question+"&appid=PK2XWK-WRQY9AH8X7")
print(ans.text)

Italian, Spanish and Portuguese form of Rudolf; from the Germanic name Hrodulf, which was derived from the elements hrod"fame" and wulf"wolf"
